### Tratamento das imagens em diferentes camadas

In [29]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(300,300,3)),

    layers.Resizing(300,300),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),           
    layers.MaxPooling2D((2,2)),  
    
    layers.Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.GlobalAveragePooling2D(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.3),

    layers.Dense(1, activation=activations.sigmoid)
])

### Configurando o optimizer

In [30]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.BinaryCrossentropy(),
    metrics = [ metrics.binary_accuracy ]
)

### Selecionando as imagens para o treinamento e teste

In [ ]:
from tensorflow.keras import utils

path = './CatsVsDogs'
batch_size = 40

train = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    seed = 1,
    subset='training',
    validation_split = 0.3,
    image_size = (300,300),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    seed = 1,
    subset='validation',
    validation_split = 0.3,
    image_size = (300,300),
    batch_size = batch_size
)

Found 2000 files belonging to 2 classes.
Using 1400 files for training.
Found 2000 files belonging to 2 classes.
Using 600 files for validation.


### Realizando o treinamento do modelo

In [32]:
from tensorflow.keras import callbacks

model_path = "./catsvsdogs.keras"

patience = 5
epochs = 100

history = model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            restore_best_weights=True,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - binary_accuracy: 0.5136 - loss: 0.6930 - val_binary_accuracy: 0.5100 - val_loss: 0.6899
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - binary_accuracy: 0.5164 - loss: 0.6905 - val_binary_accuracy: 0.6033 - val_loss: 0.6838
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - binary_accuracy: 0.5943 - loss: 0.6745 - val_binary_accuracy: 0.6000 - val_loss: 0.6532
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - binary_accuracy: 0.5893 - loss: 0.6610 - val_binary_accuracy: 0.5950 - val_loss: 0.6526
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - binary_accuracy: 0.6114 - loss: 0.6542 - val_binary_accuracy: 0.6233 - val_loss: 0.6328
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - binary_accuracy: 0.6129 - loss: 0.6458 - val_binary_accuracy: 0.6133 - val_loss: 0.6318
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - binary_accuracy: 0.6286 - loss: 0.6440 - val_binary_accuracy: 0.5900 - val_loss: 0.6501
Epoch 8/100
2